In [19]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
# from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
# from array import array
# import os
# from PIL import Image
# import sys
import time
# import os

In [20]:
'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = "5i200akRHXqb0YuDuEl7cPbt9p68P3nsm6Lxr28wG6HqqVhjWyHUJQQJ99BCACi5YpzXJ3w3AAAFACOGbqGn"
endpoint = "https://alexbalta.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

'\nEND - Authenticate\n'

In [21]:
# img = open("test1.png", "rb")
img = open("test2.jpeg", "rb")
read_response = computervision_client.read_in_stream(
    image=img,
    mode="Printed",
    raw=True
)
# print(read_response.as_dict())

operation_id = read_response.headers['Operation-Location'].split('/')[-1]
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Print the detected text, line by line
result = []
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            result.append(line.text)

print()

Lucces in resolvarea
TEMELOR la
LABORA toarele de
Inteligenta Artificialà!



In [22]:
from statistics import mean

from Levenshtein import distance as lev_dis
from Levenshtein import jaro_winkler
from jiwer import wer


def evaluate_ocr_words(ground_truth, recognized_text):
    wer_score = wer(ground_truth, recognized_text)
    return wer_score


def calculate_jaro_winkler_similarity(gt_text, ocr_text):
    return jaro_winkler(gt_text, ocr_text) * 100


def calculate_character_level_accuracy(gt_text, ocr_text):
    dist = lev_dis(gt_text, ocr_text)
    max_len = max(len(gt_text), len(ocr_text))
    accuracy = (1 - dist / max_len) * 100 if max_len > 0 else 0
    return accuracy


def calculate_word_level_accuracy(gt_text, ocr_text):
    gt_words = gt_text.split()
    ocr_words = ocr_text.split()
    common = sum(1 for w1, w2 in zip(gt_words, ocr_words) if w1 == w2)
    return (common / max(len(gt_words), len(ocr_words))) * 100 if gt_words else 0


def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return (intersection / union) * 100 if union > 0 else 0

# get/define the ground truth
# groundTruth = ["Google Cloud", "Platform"]
groundTruth = ["Succes in rezolvarea", "tEMELOR la", "LABORAtoaree de", "Inteligenta Artificiala!"]

# compute the performance
noOfCorrectLines = sum(i == j for i, j in zip(result, groundTruth))
errorLevCar = mean(calculate_character_level_accuracy(result[i],groundTruth[i]) for i in range(4))
errorJarCar = mean(calculate_jaro_winkler_similarity(result[i],groundTruth[i]) for i in range(4))
print("La nivel de caracter cu levenshtein: " + str(errorLevCar))
print("La nivel de caracter cu jaro-wrinkler: " + str(errorJarCar))

errorWord = mean(calculate_word_level_accuracy(result[i],groundTruth[i]) for i in range(4))
errorWordWer = mean(evaluate_ocr_words(result[i],groundTruth[i]) for i in range(4))
print("La nivel de cuvant: " + str(errorWord))
print("La nivel de cuvant wer: " + str(errorWordWer*100))

La nivel de caracter cu levenshtein: 91.0171568627451
La nivel de caracter cu jaro-wrinkler: 94.66176470588235
La nivel de cuvant: 33.33333333333333
La nivel de cuvant wer: 58.33333333333333


In [23]:
def calculate_iou(box1, box2):
    # Determinăm coordonatele intersecției
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    # Dacă nu există intersecție, IoU este 0
    if x1_inter >= x2_inter or y1_inter >= y2_inter:
        return 0.0

    # Calculăm aria intersecției
    intersection_area = (x2_inter - x1_inter) * (y2_inter - y1_inter)

    # Calculăm ariile fiecărei casete de delimitare
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Calculăm aria uniunii
    union_area = box1_area + box2_area - intersection_area

    # IoU = Intersecție / Uniune
    return intersection_area / union_area

# Funcție pentru procesarea imaginii și evaluarea localizării textului
def evaluate_text_localization(image_path, ground_truth_boxes):
    # Deschide fișierul imagine
    with open(image_path, "rb") as img:
        read_response = computervision_client.read_in_stream(
            image=img,
            mode="Printed",
            raw=True
        )

    operation_id = read_response.headers['Operation-Location'].split('/')[-1]
    
    # Așteptăm procesarea imaginii
    while True:
        read_result = computervision_client.get_read_result(operation_id)
        if read_result.status not in ['notStarted', 'running']:
            break
        time.sleep(1)

    # Verificăm statusul procesului și obținem rezultatele
    if read_result.status == OperationStatusCodes.succeeded:
        detected_boxes = []
        
        for text_result in read_result.analyze_result.read_results:
            for line in text_result.lines:
                # Extragem coordonatele bounding box pentru fiecare linie
                bounding_box = line.bounding_box
                detected_boxes.append([bounding_box[0],bounding_box[1],bounding_box[4],bounding_box[5]])
                print(f"Text detectat: {line.text}")
                print(f"Coordonate bounding box: {bounding_box}")

        # Calculăm IoU pentru fiecare casetă de text detectată și coordonatele de referință
        iou_scores = []
        for i in range(len(detected_boxes)):
            iou = calculate_iou(detected_boxes[i], ground_truth_boxes[i])
            iou_scores.append(iou)

        # Afișăm scorurile IoU
        for i, score in enumerate(iou_scores):
            print(f"Scor IoU {i+1}: {score:.2f}")

# Exemplu de utilizare
image_path = "test2.jpeg"

# Casete de referință (ground truth), coordonate [x1, y1, x2, y2] pentru fiecare zonă de text
ground_truth_boxes = [
    [62, 296, 1356, 480],  # Exemplu de coordonate pentru un text
    [125, 572, 1059, 724],
    [72, 925, 1022, 1036],
    [99, 1129, 1466, 1375]# Alt exemplu de coordonate pentru un alt text
]

# Evaluăm localizarea textului
evaluate_text_localization(image_path, ground_truth_boxes)

Text detectat: Lucces in resolvarea
Coordonate bounding box: [86.0, 314.0, 1335.0, 287.0, 1336.0, 443.0, 86.0, 478.0]
Text detectat: TEMELOR la
Coordonate bounding box: [140.0, 590.0, 1045.0, 587.0, 1046.0, 723.0, 140.0, 727.0]
Text detectat: LABORA toarele de
Coordonate bounding box: [81.0, 915.0, 1007.0, 926.0, 1004.0, 1039.0, 78.0, 1014.0]
Text detectat: Inteligenta Artificialà!
Coordonate bounding box: [108.0, 1129.0, 1450.0, 1151.0, 1446.0, 1293.0, 105.0, 1259.0]
Scor IoU 1: 0.68
Scor IoU 2: 0.85
Scor IoU 3: 0.87
Scor IoU 4: 0.65


In [24]:
import cv2
import numpy as np

def process_image(image_path):
    # Citește imaginea
    img = cv2.imread(image_path)

    # Convertire în grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite("gray.jpg", gray)

    # Creșterea contrastului folosind CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    contrast = clahe.apply(gray)
    cv2.imwrite("contrast.jpg", contrast)

    # Reducerea zgomotului cu Gaussian Blur
    denoise = cv2.GaussianBlur(contrast, (5, 5), 0)
    cv2.imwrite("denoise.jpg", denoise)
    
    img2 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    img2 = cv2.resize(img2, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    cv2.imwrite("improve.jpg", img2)

# Exemplu de utilizare
process_image("test2.jpeg")



In [25]:
evaluate_text_localization("gray.jpg",ground_truth_boxes)
evaluate_text_localization("denoise.jpg",ground_truth_boxes)
evaluate_text_localization("contrast.jpg",ground_truth_boxes)
evaluate_text_localization("improve.jpg",ground_truth_boxes)

Text detectat: Lucces in resolvarea
Coordonate bounding box: [87.0, 312.0, 1335.0, 289.0, 1336.0, 445.0, 87.0, 477.0]
Text detectat: TEMELOR la
Coordonate bounding box: [130.0, 592.0, 1044.0, 588.0, 1045.0, 720.0, 130.0, 726.0]
Text detectat: LABORA toarele de
Coordonate bounding box: [82.0, 916.0, 1005.0, 925.0, 1003.0, 1039.0, 79.0, 1015.0]
Text detectat: Inteligenta Artificialà!
Coordonate bounding box: [109.0, 1130.0, 1452.0, 1151.0, 1449.0, 1294.0, 106.0, 1260.0]
Scor IoU 1: 0.70
Scor IoU 2: 0.82
Scor IoU 3: 0.88
Scor IoU 4: 0.65
Text detectat: Lucces in resolvarea
Coordonate bounding box: [78.0, 337.0, 1332.0, 306.0, 1334.0, 431.0, 80.0, 473.0]
Text detectat: TEMELOR la
Coordonate bounding box: [138.0, 589.0, 1048.0, 584.0, 1049.0, 719.0, 139.0, 726.0]
Text detectat: LABORA toarele de
Coordonate bounding box: [79.0, 914.0, 1004.0, 924.0, 1003.0, 1038.0, 77.0, 1014.0]
Text detectat: Inteligenta Artificialà!
Coordonate bounding box: [103.0, 1129.0, 1454.0, 1155.0, 1450.0, 1294.0, 9